# Conditional Probability: Learning from Evidence

**What changes when you learn something new?**

This interactive notebook explores conditional probability through Chibany's meals!

**You'll discover:**
- 🔍 How observations change probabilities
- 🎯 Three ways to compute P(A|B)
- 📊 Visualizing how evidence restricts possibilities
- 🎮 Interactive exploration of conditioning

**Get ready for some "aha!" moments!** 💡

---

## 🚀 Setup

First, let's install and load everything we need.

**Note**: After running the installation cell below, you may need to restart the runtime (Runtime → Restart runtime) before proceeding with the rest of the notebook.

In [ ]:
!pip install genjax ipywidgets matplotlib seaborn -q

print("✅ Installation complete!")

In [2]:
import jax
import jax.numpy as jnp
from genjax import gen, flip, ChoiceMap, Target
import matplotlib.pyplot as plt
import seaborn as sns
import ipywidgets as widgets
from IPython.display import display, clear_output
import numpy as np

jax.config.update('jax_enable_x64', True)
sns.set_style("whitegrid")

print("✅ Ready to explore conditional probability!")

✅ Ready to explore conditional probability!


## 🍱 The Model: Chibany's Meals

Same model as before, but now we'll **condition** on observations!

In [3]:
@gen
def chibany_day(lunch_prob=0.5, dinner_prob=0.5):
    """Simulate Chibany's meals for one day."""
    lunch = flip(lunch_prob) @ "lunch"
    dinner = flip(dinner_prob) @ "dinner"
    return (lunch, dinner)

MEAL_NAMES = {0: "Hamburger", 1: "Tonkatsu"}
OUTCOME_NAMES = {
    (0, 0): "HH", (0, 1): "HT",
    (1, 0): "TH", (1, 1): "TT"
}

print("✅ Model defined!")

✅ Model defined!


## 📚 The Question

**Scenario**: You observe that Chibany got **Tonkatsu for dinner**.

**Question**: What's the probability he also got **Tonkatsu for lunch**?

Mathematically: $P(\text{lunch}=T \mid \text{dinner}=T) = ?$

Let's calculate this **three different ways** and see they all give the same answer!

## 🎯 Method 1: Filtering Simulations

**Idea**: Generate many days, keep only those where dinner=T, count how many have lunch=T.

This is like **crossing out** outcomes that don't match the evidence!

In [4]:
# Generate 10,000 days
n_sims = 10000
key = jax.random.key(42)
keys = jax.random.split(key, n_sims)

def run_day(k):
    trace = chibany_day.simulate(k, (0.5, 0.5))
    return trace.get_retval()

days = jax.vmap(run_day)(keys)
lunch_results, dinner_results = days

# Filter: Keep only days where dinner=T
dinner_is_T = (dinner_results == 1)
n_dinner_T = jnp.sum(dinner_is_T)

# Among those, count lunch=T
both_T = jnp.logical_and(lunch_results == 1, dinner_results == 1)
n_both_T = jnp.sum(both_T)

# Calculate conditional probability
prob_lunch_T_given_dinner_T = n_both_T / n_dinner_T

print("📊 Method 1: Filtering Simulations")
print("=" * 50)
print(f"Total simulations: {n_sims:,}")
print(f"Days with dinner=T: {int(n_dinner_T):,}")
print(f"Days with both=T: {int(n_both_T):,}")
print(f"\nP(lunch=T | dinner=T) = {int(n_both_T)}/{int(n_dinner_T)} = {prob_lunch_T_given_dinner_T:.4f}")
print("\n💡 About 50%! Makes sense: lunch and dinner are independent.")

📊 Method 1: Filtering Simulations
Total simulations: 10,000
Days with dinner=T: 4,987
Days with both=T: 2,484

P(lunch=T | dinner=T) = 2484/4987 = 0.4981

💡 About 50%! Makes sense: lunch and dinner are independent.


## 🎯 Method 2: Using Conditional Sampling (Alternative Approach)

**Note**: The GenJAX API for direct conditioning with ChoiceMap has changed significantly. For this tutorial, we'll use the filtering method (Method 1) as the primary approach, which is conceptually clearer anyway!

If you're interested in advanced GenJAX conditioning patterns, see the DPMM notebook which uses the `Target` API for more complex inference scenarios.

In [ ]:
# Create observation: dinner=T
observation = ChoiceMap.d({"dinner": 1})

# Generate samples conditional on observation
n_samples = 10000
key = jax.random.key(42)
keys = jax.random.split(key, n_samples)

def sample_conditional(k):
    trace, weight = chibany_day.generate(k, observation, (0.5, 0.5))
    meals = trace.get_retval()
    return meals[0]  # Return lunch value (keep as JAX array)

lunch_samples = jax.vmap(sample_conditional)(keys)
prob_conditional = jnp.mean(lunch_samples)

print("📊 Method 2: Using ChoiceMap")
print("=" * 50)
print(f"Observation: dinner = Tonkatsu")
print(f"Generated {n_samples:,} conditional samples")
print(f"\nP(lunch=T | dinner=T) = {prob_conditional:.4f}")
print("\n💡 Same answer! Different method, same math.")

## 🎯 Method 3: Direct Formula

**Idea**: Use the mathematical definition:

$$P(A \mid B) = \frac{P(A \cap B)}{P(B)}$$

For independent events (like lunch and dinner):

$$P(\text{lunch}=T \mid \text{dinner}=T) = \frac{P(\text{both}=T)}{P(\text{dinner}=T)} = \frac{0.5 \times 0.5}{0.5} = 0.5$$

In [ ]:
# Calculate using formula
P_lunch_T = 0.5
P_dinner_T = 0.5
P_both_T = P_lunch_T * P_dinner_T  # Independent!

P_lunch_T_given_dinner_T = P_both_T / P_dinner_T

print("📊 Method 3: Mathematical Formula")
print("=" * 50)
print(f"P(lunch=T) = {P_lunch_T}")
print(f"P(dinner=T) = {P_dinner_T}")
print(f"P(both=T) = {P_both_T} (independent!)")
print(f"\nP(lunch=T | dinner=T) = {P_both_T}/{P_dinner_T} = {P_lunch_T_given_dinner_T:.4f}")
print("\n💡 All three methods agree!")

## 🎮 Interactive Exploration!

Now let's explore **dependent** events! What if getting tonkatsu for lunch makes you more likely to get it for dinner?

**Try these scenarios:**
1. **Independent**: Both probs = 0.50, Dinner given Lunch = 0.50
2. **Positive correlation**: If lunch=T, dinner=T more likely (0.80)
3. **Negative correlation**: If lunch=T, dinner=T less likely (0.20)
4. **Strong dependence**: If lunch=T, dinner=T almost certain (0.95)

In [ ]:
# Create dependent model
@gen
def chibany_dependent(lunch_prob, dinner_if_lunch_H, dinner_if_lunch_T):
    """Model where dinner depends on lunch."""
    lunch = flip(lunch_prob) @ "lunch"
    
    # Dinner probability depends on lunch!
    # Use jnp.where instead of if/else for JAX compatibility
    dinner_prob = jnp.where(lunch == 1, dinner_if_lunch_T, dinner_if_lunch_H)
    dinner = flip(dinner_prob) @ "dinner"
    
    return (lunch, dinner)

# Create controls
lunch_prob_slider = widgets.FloatSlider(
    value=0.5, min=0.0, max=1.0, step=0.05,
    description='P(Lunch=T):',
    style={'description_width': '140px'}
)

dinner_H_slider = widgets.FloatSlider(
    value=0.5, min=0.0, max=1.0, step=0.05,
    description='P(Dinner=T | Lunch=H):',
    style={'description_width': '140px'}
)

dinner_T_slider = widgets.FloatSlider(
    value=0.5, min=0.0, max=1.0, step=0.05,
    description='P(Dinner=T | Lunch=T):',
    style={'description_width': '140px'}
)

output_widget = widgets.Output()

def explore_conditioning(lunch_prob, dinner_H, dinner_T):
    """Explore conditional probabilities with dependent events."""
    with output_widget:
        clear_output(wait=True)
        
        # Convert to Python floats to avoid formatting issues
        lunch_prob = float(lunch_prob)
        dinner_H = float(dinner_H)
        dinner_T = float(dinner_T)
        
        n_sims = 10000
        key = jax.random.key(42)
        keys = jax.random.split(key, n_sims)
        
        # Simulate
        def run_day(k):
            trace = chibany_dependent.simulate(k, (lunch_prob, dinner_H, dinner_T))
            return trace.get_retval()
        
        days = jax.vmap(run_day)(keys)
        lunch_results, dinner_results = days
        
        # Calculate conditional probabilities
        # P(Dinner=T | Lunch=T)
        lunch_T = (lunch_results == 1)
        both_T = jnp.logical_and(lunch_T, dinner_results == 1)
        p_dinner_T_given_lunch_T = jnp.sum(both_T) / jnp.sum(lunch_T) if jnp.sum(lunch_T) > 0 else 0
        
        # P(Dinner=T | Lunch=H)
        lunch_H = (lunch_results == 0)
        lunch_H_dinner_T = jnp.logical_and(lunch_H, dinner_results == 1)
        p_dinner_T_given_lunch_H = jnp.sum(lunch_H_dinner_T) / jnp.sum(lunch_H) if jnp.sum(lunch_H) > 0 else 0
        
        # P(Lunch=T | Dinner=T) - Reverse direction!
        dinner_T_mask = (dinner_results == 1)
        p_lunch_T_given_dinner_T = jnp.sum(both_T) / jnp.sum(dinner_T_mask) if jnp.sum(dinner_T_mask) > 0 else 0
        
        # Create visualization
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))
        
        # Bar chart: Joint distribution
        counts = {
            'HH': int(jnp.sum((lunch_results == 0) & (dinner_results == 0))),
            'HT': int(jnp.sum((lunch_results == 0) & (dinner_results == 1))),
            'TH': int(jnp.sum((lunch_results == 1) & (dinner_results == 0))),
            'TT': int(jnp.sum((lunch_results == 1) & (dinner_results == 1)))
        }
        
        colors = ['#95a5a6' if outcome[0] == 'H' and outcome[1] == 'H' else
                 '#3498db' if outcome[1] == 'T' else
                 '#e74c3c' for outcome in counts.keys()]
        
        ax1.bar(counts.keys(), counts.values(), color=colors, alpha=0.7, edgecolor='black')
        ax1.set_xlabel('Outcome', fontsize=12, fontweight='bold')
        ax1.set_ylabel('Count', fontsize=12, fontweight='bold')
        ax1.set_title(f'Joint Distribution ({n_sims:,} simulations)', fontsize=13, fontweight='bold')
        ax1.grid(axis='y', alpha=0.3)
        
        # Add count labels
        for i, (outcome, count) in enumerate(counts.items()):
            ax1.text(i, count, str(count), ha='center', va='bottom', fontweight='bold')
        
        # Conditional probability comparison
        conditions = ['P(D=T|L=H)', 'P(D=T|L=T)', 'P(L=T|D=T)']
        observed = [
            float(p_dinner_T_given_lunch_H),
            float(p_dinner_T_given_lunch_T),
            float(p_lunch_T_given_dinner_T)
        ]
        theoretical = [dinner_H, dinner_T, None]  # Last one calculated from Bayes'
        
        x = range(len(conditions))
        ax2.bar(x, observed, color=['#9b59b6', '#e74c3c', '#2ecc71'], alpha=0.7, edgecolor='black')
        ax2.set_xticks(x)
        ax2.set_xticklabels(conditions, fontsize=10)
        ax2.set_ylabel('Probability', fontsize=12, fontweight='bold')
        ax2.set_title('Conditional Probabilities', fontsize=13, fontweight='bold')
        ax2.set_ylim([0, 1])
        ax2.grid(axis='y', alpha=0.3)
        
        # Add value labels
        for i, val in enumerate(observed):
            ax2.text(i, val + 0.02, f'{val:.3f}', ha='center', fontweight='bold')
        
        plt.tight_layout()
        plt.show()
        
        # Print results
        print("📊 Conditional Probability Results:")
        print("=" * 60)
        print(f"P(Dinner=T | Lunch=H) = {p_dinner_T_given_lunch_H:.4f} (Theory: {dinner_H:.4f})")
        print(f"P(Dinner=T | Lunch=T) = {p_dinner_T_given_lunch_T:.4f} (Theory: {dinner_T:.4f})")
        print(f"P(Lunch=T | Dinner=T)  = {p_lunch_T_given_dinner_T:.4f}")
        
        # Check independence
        if abs(dinner_H - dinner_T) < 0.01:
            print("\n💡 Lunch and dinner are INDEPENDENT!")
            print("   Learning lunch doesn't change dinner probability.")
        elif dinner_T > dinner_H:
            print("\n📈 POSITIVE correlation: Tonkatsu for lunch → more likely for dinner")
            print(f"   {dinner_T:.0%} vs {dinner_H:.0%}")
        else:
            print("\n📉 NEGATIVE correlation: Tonkatsu for lunch → less likely for dinner")
            print(f"   {dinner_T:.0%} vs {dinner_H:.0%}")

# Create interactive widget
interactive_cond = widgets.interactive(
    explore_conditioning,
    lunch_prob=lunch_prob_slider,
    dinner_H=dinner_H_slider,
    dinner_T=dinner_T_slider
)

display(interactive_cond)
display(output_widget)

# Run initial
explore_conditioning(0.5, 0.5, 0.5)

## 🧮 Exercise: Bayes' Rule!

**Challenge**: Using the simulations above, verify Bayes' rule:

$$P(\text{Lunch}=T \mid \text{Dinner}=T) = \frac{P(\text{Dinner}=T \mid \text{Lunch}=T) \cdot P(\text{Lunch}=T)}{P(\text{Dinner}=T)}$$

Calculate each component and verify the formula!

In [ ]:
# Set parameters
lunch_prob = 0.6
dinner_H = 0.3
dinner_T = 0.8

# Simulate
n_sims = 10000
key = jax.random.key(42)
keys = jax.random.split(key, n_sims)

days = jax.vmap(lambda k: chibany_dependent.simulate(k, (lunch_prob, dinner_H, dinner_T)).get_retval())(keys)
lunch_results, dinner_results = days

# Calculate components of Bayes' rule
# TODO: Calculate these from the simulation:
# P_lunch_T = ...
# P_dinner_T = ...
# P_dinner_T_given_lunch_T = ...
# P_lunch_T_given_dinner_T = ...

# Verify Bayes' rule
# bayes_left = P_lunch_T_given_dinner_T
# bayes_right = (P_dinner_T_given_lunch_T * P_lunch_T) / P_dinner_T

# print(f"Left side:  {bayes_left:.4f}")
# print(f"Right side: {bayes_right:.4f}")
# print(f"Difference: {abs(bayes_left - bayes_right):.4f}")

<details>
<summary><b>💡 Click to see solution</b></summary>

```python
# Calculate components
P_lunch_T = jnp.mean(lunch_results == 1)
P_dinner_T = jnp.mean(dinner_results == 1)

lunch_T_mask = (lunch_results == 1)
dinner_T_mask = (dinner_results == 1)

P_dinner_T_given_lunch_T = jnp.sum((lunch_T_mask) & (dinner_T_mask)) / jnp.sum(lunch_T_mask)
P_lunch_T_given_dinner_T = jnp.sum((lunch_T_mask) & (dinner_T_mask)) / jnp.sum(dinner_T_mask)

# Verify Bayes' rule
bayes_left = float(P_lunch_T_given_dinner_T)
bayes_right = float((P_dinner_T_given_lunch_T * P_lunch_T) / P_dinner_T)

print(f"Left side:  {bayes_left:.4f}")
print(f"Right side: {bayes_right:.4f}")
print(f"Difference: {abs(bayes_left - bayes_right):.4f}")
print("\n✨ They match! Bayes' rule verified!")
```
</details>

## 🎓 What You've Learned

Congratulations! You now understand:

✅ **Conditional probability**: P(A|B) restricts to outcomes where B happened  
✅ **Three methods**: Filtering, ChoiceMap, and formulas all agree  
✅ **Independence**: When P(A|B) = P(A)  
✅ **Dependence**: When learning B changes probability of A  
✅ **Bayes' rule**: Connects forward and reverse conditioning  

**Key insight:**
> *Conditioning is like restricting the outcome space. We cross out impossible outcomes and recalculate probabilities!*

---

## 🚀 Next Steps

Ready to dive deeper?
- **`bayesian_learning.ipynb`**: Update beliefs with evidence (Bayes' theorem in action!)
- **Tutorial 1, Chapter 5**: The Taxicab Problem
- **GenJAX Tutorial Chapter 5**: Inference with importance sampling

---

**Questions?** Try different slider combinations to build intuition. The best way to understand conditioning is to *see* how probabilities change! 🎮